# Afficher les 5 principales villes en fonction des ventes
- Visualisation: Ventes par produit + comparaison année précédente

In [ ]:
# Ajouter les colonnes calculées si ce n'est pas encore fait
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Lire le fichier
df = pd.read_excel("sales.xlsx")

# Créer les colonnes calculées correctement
df["Sales"] = df["Order Quantity"] * df["Unit Selling Price"]
df["Cost"] = df["Order Quantity"] * df["Unit Cost"]
df["Profit"] = df["Sales"] - df["Cost"]

# Créer la table de dates à partir de OrderDate
date_range = pd.date_range(start=df["OrderDate"].min(), end=df["OrderDate"].max(), freq='D')
date_table = pd.DataFrame({"Date": date_range})

date_table["Year"] = date_table["Date"].dt.year
date_table["Month"] = date_table["Date"].dt.month
date_table["Month Name"] = date_table["Date"].dt.strftime('%B')
date_table["Quarter"] = date_table["Date"].dt.quarter
date_table["Day"] = date_table["Date"].dt.day
date_table["Day Name"] = date_table["Date"].dt.strftime('%A')
date_table["Week"] = date_table["Date"].dt.isocalendar().week

# Renommer la colonne pour correspondre à OrderDate
date_table_renamed = date_table.rename(columns={"Date": "OrderDate"})
# Fusion
df_merged = pd.merge(df, date_table_renamed, on="OrderDate", how="left")

# Top 5 des régions (ou villes) par ventes
top5_regions = df.groupby("Delivery Region Index").agg({
    "Sales": "sum"
}).sort_values(by="Sales", ascending=False).head(5).reset_index()

# Affichage
print("Top 5 régions par ventes :")
print(top5_regions)

plt.figure(figsize=(8, 5))
sns.barplot(data=top5_regions, x="Delivery Region Index", y="Sales", palette="viridis")
plt.title("Top 5 régions par ventes")
plt.xlabel("Région (Index)")
plt.ylabel("Ventes")
plt.tight_layout()
plt.show()

# Comparer les bénéfices par canal de vente (Channel) avec ceux de l’année précédente
print("\nComparaison des bénéfices par canal de vente :")
# Regrouper par canal et année
channel_profit = df_merged.groupby(["Channel", "Year"]).agg({
    "Profit": "sum"
}).reset_index()

# Trier les données par canal et année pour appliquer le shift
channel_profit = channel_profit.sort_values(by=["Channel", "Year"])

# Calcul de la colonne "Profit PY"
channel_profit["Profit PY"] = channel_profit.groupby("Channel")["Profit"].shift(1)

# Calcul de la variation absolue et en pourcentage
channel_profit["Profit Var"] = channel_profit["Profit"] - channel_profit["Profit PY"]
channel_profit["Profit Var %"] = (channel_profit["Profit Var"] / channel_profit["Profit PY"]) * 100

# Affichage
print(channel_profit.round(2))
